# face-centered data examples

The `FaceCenterData2d` represents data that is centered on faces.  Here we explore the methods provided

In [1]:
import numpy as np
import pyro.mesh.boundary as bnd
import pyro.mesh.patch as patch
import matplotlib.pyplot as plt
%matplotlib inline

# for unit testing, we want to ensure the same random numbers
np.random.seed(100)

In [2]:
g = patch.Grid2d(4, 6, ng=2)
print(g)

2-d grid: nx = 4, ny = 6, ng = 2


In [3]:
bc = bnd.BC(xlb="periodic", xrb="periodic",
            ylb="periodic", yrb="periodic")
print(bc)

BCs: -x: periodic  +x: periodic  -y: periodic  +y: periodic


In [4]:
# create data that is centered on x-faces
d = patch.FaceCenterData2d(g, 1)
d.register_var("a", bc)
d.create()
print(d)

fc data: idir = 1, nx = 4, ny = 6, ng = 2
         nvars = 1
         variables:
               a: min:    0.0000000000    max:    0.0000000000
                  BCs: -x: periodic     +x: periodic     -y: periodic     +y: periodic    



Now we fill the grid with random data.  `get_var()` returns an `ArrayIndexer` object that has methods for accessing views into the data.  Here we use `a.v()` to get the "valid" region, i.e. excluding ghost cells.

In [5]:
a = d.get_var("a")
a.v()[:,:] = np.random.rand(g.nx+1, g.ny)

In [6]:
type(a)

pyro.mesh.array_indexer.ArrayIndexerFC

In [7]:
# since we are nodal in x, it should be the case that ilo and ihi+1 are identical
a[a.g.ihi+1,:] = a[a.g.ilo,:]

when we pretty_print() the variable, we see the ghost cells colored red.  Note that we just filled the interior above.

In [8]:
a.pretty_print()

         0         0         0         0         0         0         0         0         0 
         0         0         0         0         0         0         0         0         0 
         0         0   0.12157    0.2092   0.17194   0.33611   0.12157         0         0 
         0         0 0.0047189   0.89132   0.81168   0.81765 0.0047189         0         0 
         0         0   0.84478   0.57509   0.97862   0.94003   0.84478         0         0 
         0         0   0.42452   0.13671    0.2197    0.4317   0.42452         0         0 
         0         0   0.27837   0.82585   0.10838   0.27407   0.27837         0         0 
         0         0    0.5434   0.67075   0.18533   0.81622    0.5434         0         0 
         0         0         0         0         0         0         0         0         0 
         0         0         0         0         0         0         0         0         0 

         ^ y
         |
         +---> x
        


`pretty_print()` can also take an argument, specifying the format string to be used for the output.

In [9]:
a.pretty_print(fmt="%7.3g")

      0      0      0      0      0      0      0      0      0 
      0      0      0      0      0      0      0      0      0 
      0      0  0.122  0.209  0.172  0.336  0.122      0      0 
      0      00.00472  0.891  0.812  0.8180.00472      0      0 
      0      0  0.845  0.575  0.979   0.94  0.845      0      0 
      0      0  0.425  0.137   0.22  0.432  0.425      0      0 
      0      0  0.278  0.826  0.108  0.274  0.278      0      0 
      0      0  0.543  0.671  0.185  0.816  0.543      0      0 
      0      0      0      0      0      0      0      0      0 
      0      0      0      0      0      0      0      0      0 

         ^ y
         |
         +---> x
        


now fill the ghost cells -- notice that the left and right are periodic, the upper is outflow, and the lower is reflect, as specified when we registered the data above.

In [10]:
d.fill_BC("a")
a.pretty_print()

   0.10838   0.27407   0.27837   0.82585   0.10838   0.27407   0.27837   0.82585   0.10838 
   0.18533   0.81622    0.5434   0.67075   0.18533   0.81622    0.5434   0.67075   0.18533 
   0.17194   0.33611   0.12157    0.2092   0.17194   0.33611   0.12157    0.2092   0.17194 
   0.81168   0.81765 0.0047189   0.89132   0.81168   0.81765 0.0047189   0.89132   0.81168 
   0.97862   0.94003   0.84478   0.57509   0.97862   0.94003   0.84478   0.57509   0.97862 
    0.2197    0.4317   0.42452   0.13671    0.2197    0.4317   0.42452   0.13671    0.2197 
   0.10838   0.27407   0.27837   0.82585   0.10838   0.27407   0.27837   0.82585   0.10838 
   0.18533   0.81622    0.5434   0.67075   0.18533   0.81622    0.5434   0.67075   0.18533 
   0.17194   0.33611   0.12157    0.2092   0.17194   0.33611   0.12157    0.2092   0.17194 
   0.81168   0.81765 0.0047189   0.89132   0.81168   0.81765 0.0047189   0.89132   0.81168 

         ^ y
         |
         +---> x
        


We can find the L2 norm of the data easily

In [11]:
print("{:12.6g}".format(a.norm()))

    0.619671


and the min and max

In [12]:
print(a.min(), a.max())

0.004718856190972565 0.9786237847073697


## `ArrayIndexer`

We we access the data, an `ArrayIndexer` object is returned.  The `ArrayIndexer` sub-classes the NumPy `ndarray`, so it can do all of the methods that a NumPy array can, but in addition, we can use the `ip()`, `jp()`, or `ipjp()` methods to the `ArrayIndexer` object shift our view in the x, y, or x & y directions.

To make this clearer, we'll change our data set to be nicely ordered numbers.  We index the `ArrayIndex` the same way we would a NumPy array.  The index space includes ghost cells, so the `ilo` and `ihi` attributes from the grid object are useful to index just the valid region.  The `.v()` method is a shortcut that also gives a view into just the valid data.

Note: when we use one of the `ip()`, `jp()`, `ipjp()`, or `v()` methods, the result is a regular NumPy `ndarray`, not an `ArrayIndexer` object.  This is because it only spans part of the domain (e.g., no ghost cells), and therefore cannot be associated with the `Grid2d` object that the `ArrayIndexer` is built from.

In [13]:
type(a)

pyro.mesh.array_indexer.ArrayIndexerFC

In [14]:
type(a.v())

numpy.ndarray

In [15]:
a[:,:] = np.arange((g.qx+1)*g.qy).reshape(g.qx+1, g.qy)

In [16]:
a.pretty_print()

         9        19        29        39        49        59        69        79        89 
         8        18        28        38        48        58        68        78        88 
         7        17        27        37        47        57        67        77        87 
         6        16        26        36        46        56        66        76        86 
         5        15        25        35        45        55        65        75        85 
         4        14        24        34        44        54        64        74        84 
         3        13        23        33        43        53        63        73        83 
         2        12        22        32        42        52        62        72        82 
         1        11        21        31        41        51        61        71        81 
         0        10        20        30        40        50        60        70        80 

         ^ y
         |
         +---> x
        


We index our arrays as {i,j}, so x (indexed by i) is the row and y (indexed by j) is the column in the NumPy array.   Note that python arrays are stored in row-major order, which means that all of the entries in the same row are adjacent in memory.  This means that when we simply print out the `ndarray`, we see constant-x horizontally, which is the transpose of what we are used to.

In [17]:
a.v()

array([[22., 23., 24., 25., 26., 27.],
       [32., 33., 34., 35., 36., 37.],
       [42., 43., 44., 45., 46., 47.],
       [52., 53., 54., 55., 56., 57.],
       [62., 63., 64., 65., 66., 67.]])

We can offset our view into the array by one in x -- this would be like {i+1, j} when we loop over data.  The `ip()` method is used here, and takes an argument which is the (positive) shift in the x (i) direction.  So here's a shift by 1

In [18]:
a.ip(-1, buf=1)

array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
       [11., 12., 13., 14., 15., 16., 17., 18.],
       [21., 22., 23., 24., 25., 26., 27., 28.],
       [31., 32., 33., 34., 35., 36., 37., 38.],
       [41., 42., 43., 44., 45., 46., 47., 48.],
       [51., 52., 53., 54., 55., 56., 57., 58.],
       [61., 62., 63., 64., 65., 66., 67., 68.]])

A shifted view is necessarily smaller than the original array, and relies on ghost cells to bring new data into view.  Because of this, the underlying data is no longer the same size as the original data, so we return it as an `ndarray` (which is actually just a view into the data in the `ArrayIndexer` object, so no copy is made.

To see that it is simply a view, lets shift and edit the data

In [19]:
d = a.ip(1)
d[1,1] = 0.0
a.pretty_print()

         9        19        29        39        49        59        69        79        89 
         8        18        28        38        48        58        68        78        88 
         7        17        27        37        47        57        67        77        87 
         6        16        26        36        46        56        66        76        86 
         5        15        25        35        45        55        65        75        85 
         4        14        24        34        44        54        64        74        84 
         3        13        23        33         0        53        63        73        83 
         2        12        22        32        42        52        62        72        82 
         1        11        21        31        41        51        61        71        81 
         0        10        20        30        40        50        60        70        80 

         ^ y
         |
         +---> x
        


Here, since d was really a view into $a_{i+1,j}$, and we accessed element (1,1) into that view (with 0,0 as the origin), we were really accessing the element (2,1) in the valid region